In [37]:

import nni.retiarii.nn.pytorch as nn
import torch
import logging

from nni.retiarii import model_wrapper

from collections import OrderedDict


logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class Convolutions(nn.Module):
    def __init__(self, conv, layer_name):
        super().__init__()

        self.conv = nn.LayerChoice(conv, label=f'{layer_name} - Step 2: Convolutions, Batchnorm and Activation')

    def forward(self, x):
        x = self.conv(x)
        return x
    
class BaseBlock(nn.Module):
    def __init__(self):
        super(BaseBlock, self).__init__()

    def get_conv_ordered_dict(self, in_channels, out_channels, ks, pd, dl, activation):
        layers = [
            ("Conv2d", nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=ks, padding=pd, dilation=dl),
                nn.BatchNorm2d(out_channels),
                activation,
                nn.Conv2d(out_channels, out_channels, kernel_size=ks, padding=pd, dilation=dl),
                nn.BatchNorm2d(out_channels),
                activation
                )
            ),
            ("DepthwiseSeparable", nn.Sequential(
                nn.Conv2d(in_channels, in_channels, kernel_size=ks, padding=pd, dilation=dl, groups=in_channels),
                nn.Conv2d(in_channels, out_channels, kernel_size=1),
                nn.BatchNorm2d(out_channels),
                activation,
                nn.Conv2d(out_channels, out_channels, kernel_size=ks, padding=pd, dilation=dl, groups=out_channels),
                nn.Conv2d(out_channels, out_channels, kernel_size=1),
                nn.BatchNorm2d(out_channels),
                activation
                )
            )
        ]
        return OrderedDict(layers)

    def crop_tensor(self, target_tensor, tensor):
        target_size = target_tensor.size()[2]  # Assuming height and width are same
        tensor_size = tensor.size()[2]
        delta = tensor_size - target_size
        delta = delta // 2
        return tensor[:, :, delta:tensor_size-delta, delta:tensor_size-delta]

class EncoderBlock(BaseBlock):
    def __init__(self, in_channels, out_channels, ks, pd, dl, activations, downsamples, layer_name):
        super(EncoderBlock, self).__init__()
        
        self.downsample = downsamples
        self.conv_layer = Convolutions(self.get_conv_ordered_dict(in_channels, out_channels, ks, pd, dl, activations), layer_name)

    def forward(self, x):
        x = self.downsample(x)
        x = self.conv_layer(x)
        return x

class DecoderBlock(BaseBlock):
    def __init__(self, in_channels, out_channels, ks, pd, dl, activations, upsamples, layer_name):
        super(DecoderBlock, self).__init__()

        self.upsample = upsamples
        self.conv_layer = Convolutions(self.get_conv_ordered_dict(in_channels, out_channels, ks, pd, dl, activations), layer_name)

    def forward(self, x, skip):
        upsampled = self.upsample(x)
        cropped = self.crop_tensor(upsampled, skip)
        return self.conv_layer(torch.cat([cropped, upsampled], 1))

@model_wrapper
class SearchSpace(BaseBlock):
    def __init__(self, in_channels=1, out_channels=1):
        super().__init__()
        ks = 5
        dl = 3
        pd = (ks - 1) * dl // 2

        activation = nn.SiLU(inplace=True)

        self.downsamples = nn.MaxPool2d(kernel_size=2, stride=2)

        self.upsamples = nn.Upsample(scale_factor=2,mode='nearest')

        # Conv layer in"
        self.mid_channels = 64
        self.first = nn.Sequential(
                nn.Conv2d(in_channels, self.mid_channels, kernel_size=ks, padding=pd, dilation=dl),
                nn.BatchNorm2d(self.mid_channels),
                activation,
                nn.Conv2d(self.mid_channels, self.mid_channels, kernel_size=ks, padding=pd, dilation=dl),
                nn.BatchNorm2d(self.mid_channels),
                activation
                )

        # Conv layer out
        self.out = nn.Conv2d(self.mid_channels, out_channels, kernel_size=1, padding=0, dilation=1)
        
    def forward(self, x):
        x = self.first(x)

        x = self.out(x)
        return x

In [31]:
import nni
import torch
import nni.retiarii.strategy as strategy

from darts.eval import main_evaluation

from nni.experiment import Experiment
from nni.retiarii.evaluator import FunctionalEvaluator
from nni.retiarii.experiment.pytorch import RetiariiExperiment, RetiariiExeConfig


In [44]:

# search space
model_space = SearchSpace()
evaluator = FunctionalEvaluator(main_evaluation)

# search strategy
# search_strategy = strategy.Random(dedup=True)
search_strategy = strategy.DARTS()
# experiment
exp = RetiariiExperiment(model_space, evaluator, [], search_strategy)
exp_config = RetiariiExeConfig('local')
exp_config.experiment_name = 'mnist_search'
exp_config.trial_code_directory = 'C:/Users/Public/Public_VS_Code/NAS_test'
exp_config.experiment_working_directory = 'C:/Users/Public/nni-experiments'

exp_config.max_trial_number = 12   # spawn 50 trials at most
exp_config.trial_concurrency = 2  # will run two trials concurrently

exp_config.trial_gpu_number = 1 # will run 1 trial(s) concurrently
exp_config.training_service.use_active_gpu = True

# Execute
exp.run(exp_config, 8081)

[2023-08-16 18:14:58] Dispatcher terminiated


2023-08-16 18:14:58,685 - INFO - Dispatcher terminiated


[2023-08-16 18:14:58] Creating experiment, Experiment ID: hnm0w49y


2023-08-16 18:14:58,796 - INFO - Creating experiment, Experiment ID: ${CYAN}hnm0w49y


[2023-08-16 18:14:58] Starting web server...


...

[2023-08-16 18:15:00] Start strategy...


2023-08-16 18:15:00,057 - INFO - Start strategy...


[2023-08-16 18:15:00] Successfully update searchSpace.


2023-08-16 18:15:00,094 - INFO - Successfully update searchSpace.


TypeError: Evaluator needs to be a lightning evaluator to make one-shot strategy work.

In [45]:
experiment = Experiment.connect(8081)
experiment.stop()

[2023-08-16 18:17:16] Connect to port 8081 success, experiment id is hnm0w49y, status is RUNNING.


2023-08-16 18:17:16,707 - INFO - Connect to port 8081 success, experiment id is hnm0w49y, status is RUNNING.


[2023-08-16 18:17:16] Stopping experiment, please wait...


2023-08-16 18:17:16,708 - INFO - Stopping experiment, please wait...


[2023-08-16 18:17:16] Experiment stopped


2023-08-16 18:17:16,729 - INFO - Experiment stopped


[2023-08-16 18:17:16] Dispatcher exiting...


2023-08-16 18:17:16,737 - INFO - Dispatcher exiting...


[2023-08-16 18:17:18] Dispatcher terminiated


2023-08-16 18:17:18,494 - INFO - Dispatcher terminiated
